In [73]:
# import, configuration, functions, etc.
# Run this every time you use the script

import json
import requests
from time import sleep
import csv
import datetime

sparqlSleep = 0.1 # number of seconds to wait between queries to SPARQL endpoint

def generateHeaderDictionary(acceptMediaType):
    userAgentHeader = 'VanderBot/1.0 (https://github.com/HeardLibrary/linked-data/tree/master/publications; mailto:steve.baskauf@vanderbilt.edu)'
    requestHeaderDictionary = {
        'Accept' : acceptMediaType,
        'User-Agent': userAgentHeader
    }
    return requestHeaderDictionary

def generate_utc_date():
    wholeTimeStringZ = datetime.datetime.utcnow().isoformat() # form: 2019-12-05T15:35:04.959311
    dateZ = wholeTimeStringZ.split('T')[0] # form 2019-12-05
    return dateZ

# extracts the qNumber from a Wikidata IRI
def extract_qnumber(iri):
    # pattern is http://www.wikidata.org/entity/Q6386232
    pieces = iri.split('/')
    return pieces[4]

# write a list of lists to a CSV file
def writeListsToCsv(fileName, array):
    with open(fileName, 'w', newline='', encoding='utf-8') as fileObject:
        writerObject = csv.writer(fileObject)
        for row in array:
            writerObject.writerow(row)


def read_lists_from_csv(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as file_object:
        reader_object = csv.reader(file_object)
        list_of_lists = []
        for row_list in reader_object:
            list_of_lists.append(row_list)
    return list_of_lists


            # write a list of dictionaries to a CSV file
def writeDictsToCsv(table, filename, fieldnames):
    with open(filename, 'w', newline='', encoding='utf-8') as csvFileObject:
        writer = csv.DictWriter(csvFileObject, fieldnames=fieldnames)
        writer.writeheader()
        for row in table:
            writer.writerow(row)

            # read from a CSV file into a list of dictionaries
def readDict(filename):
    with open(filename, 'r', newline='', encoding='utf-8') as fileObject:
        dictObject = csv.DictReader(fileObject)
        array = []
        for row in dictObject:
            array.append(row)
    return array

def get_vu_counts(query):
    wikidataEndpointUrl = 'https://query.wikidata.org/sparql'
    acceptMediaType = 'application/json'
    r = requests.get(wikidataEndpointUrl, params={'query' : query}, headers = generateHeaderDictionary(acceptMediaType))
    try:
        data = r.json()
        #print(json.dumps(data, indent=2))
        count = data['results']['bindings'][0]['count']['value']
    except:
        count = [r.text]
    # delay to avoid hitting the SPARQL endpoint to rapidly
    sleep(sparqlSleep)
    return count

def get_unit_counts(query):
    table = []
    wikidataEndpointUrl = 'https://query.wikidata.org/sparql'
    acceptMediaType = 'application/json'
    r = requests.get(wikidataEndpointUrl, params={'query' : query}, headers = generateHeaderDictionary(acceptMediaType))
    try:
        data = r.json()
        statements = data['results']['bindings']
        for statement in statements:
            unit_iri = statement['unit']['value']
            unit_qnumber = extract_qnumber(unit_iri)
            count = statement['count']['value']
            table.append({'unit': unit_qnumber, 'count': count})
    except:
        table = [r.text]
    # delay to avoid hitting the SPARQL endpoint to rapidly
    sleep(sparqlSleep)
    return table

def add_query_to_vu_table(filename, query):
    table = read_lists_from_csv(filename)
    #print(table)

    count = get_vu_counts(query)
    #print(count)
    
    date = generate_utc_date()
    row_list = [date]
    row_list.append(count)
    #print(row_list)
    table.append(row_list)
    #print(table)

    writeListsToCsv(filename, table)

def add_query_to_unit_table(filename, query):
    table = read_lists_from_csv(filename)
    #print(table)

    dictionary = get_unit_counts(query)
    #print(json.dumps(dictionary, indent=2))
    
    date = generate_utc_date()
    row_list = [date]
    for header in table[0][1:len(table[0])]: # skip the first item (date)
        found = False
        for count in dictionary:
            if count['unit'] == header:
                found = True
                row_list.append(count['count'])
        if not found:
            row_list.append('0')
    #print(row_list)
    table.append(row_list)

    writeListsToCsv(filename, table)
    
def run_all_queries():
    # -----------------------------
    # Queries for all of Vanderbilt
    # -----------------------------

    # query to get the total number of persons affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?person) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      }
    '''
    filename = 'vu_total.csv'
    add_query_to_vu_table(filename, query)
    print(filename)

    # query to get the total number of men affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?man) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?man wdt:P1416 ?unit.
      ?man wdt:P21 wd:Q6581097.
      }
    '''
    filename = 'vu_men.csv'
    add_query_to_vu_table(filename, query)
    print(filename)

    # query to get the total number of women affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?woman) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?woman wdt:P1416 ?unit.
      ?woman wdt:P21 wd:Q6581072.
      }
    '''
    filename = 'vu_women.csv'
    add_query_to_vu_table(filename, query)
    print(filename)

    # query to count number of people with ORCIDs
    query = '''
    select (count(distinct ?person) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      ?person wdt:P496 ?orcid.
      }
    '''
    filename = 'vu_orcid.csv'
    add_query_to_vu_table(filename, query)
    print(filename)
    
    # query to get the total number works authored by anyone affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      ?work wdt:P50 ?person.
      }
    '''
    filename = 'vu_works.csv'
    add_query_to_vu_table(filename, query)
    print(filename)

    # query to get the total number works authored by men affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?man wdt:P1416 ?unit.
      ?man wdt:P21 wd:Q6581097.
      ?work wdt:P50 ?man.
      }
    '''
    filename = 'vu_men_works.csv'
    add_query_to_vu_table(filename, query)
    print(filename)

    # query to get the total number works authored by women affiliated with Vanderbilt units
    query = '''
    select (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?woman wdt:P1416 ?unit.
      ?woman wdt:P21 wd:Q6581072.
      ?work wdt:P50 ?woman.
      }
    '''
    filename = 'vu_women_works.csv'
    add_query_to_vu_table(filename, query)
    print(filename)



    # ------------------------
    # Query by Vanderbilt unit
    # ------------------------

    # query to get the total number of persons affiliated with each unit
    query = '''
    select ?unit (count(distinct ?person) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      }
    group by ?unit
    '''
    filename = 'units_total.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of women affiliated with each unit
    query = '''
    select ?unit (count(distinct ?woman) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?woman wdt:P1416 ?unit.
      ?woman wdt:P21 wd:Q6581072.
      }
    group by ?unit
    '''
    filename = 'units_women.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of men affiliated with each unit
    query = '''
    select ?unit (count(distinct ?man) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?man wdt:P1416 ?unit.
      ?man wdt:P21 wd:Q6581097.
      }
    group by ?unit
    '''
    filename = 'units_men.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of people having ORCIDs affiliated with each unit
    query = '''
    select ?unit (count(distinct ?person) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      ?person wdt:P496 ?orcid.
      }
    group by ?unit
    '''
    filename = 'units_orcid.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of authored works associated with each unit
    query = '''
    select ?unit (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?person wdt:P1416 ?unit.
      ?work wdt:P50 ?person.
      }
    group by ?unit
    '''
    filename = 'units_works.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of authored works by men affiliated with each unit
    query = '''
    select ?unit (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?man wdt:P1416 ?unit.
      ?man wdt:P21 wd:Q6581097.
      ?work wdt:P50 ?man.
      }
    group by ?unit
    '''
    filename = 'units_works_men.csv'
    add_query_to_unit_table(filename, query)
    print(filename)

    # query to get the total number of authored works by women affiliated with each unit
    query = '''
    select ?unit (count(distinct ?work) as ?count)  where {
      ?unit wdt:P749+ wd:Q29052.
      ?woman wdt:P1416 ?unit.
      ?woman wdt:P21 wd:Q6581072.
      ?work wdt:P50 ?woman.
      }
    group by ?unit
    '''
    filename = 'units_works_women.csv'
    add_query_to_unit_table(filename, query)
    print(filename)



In [74]:
# ---------------
# Run this once the first time you set up
# !!! Running it again will wipe out all of your data !!!
# ---------------

# Get the Wikidata IDs and names for Vanderbilt Units

# create a string for the query
query = '''
select ?label ?unit ?parent where {
  ?unit wdt:P749+ wd:Q29052.  # Q29052 is Vanderbilt
  ?unit wdt:P749 ?parent.
  ?unit rdfs:label ?label.
  filter(lang(?label) = 'en')
  }
order by ?label
'''

#print(query)

unit_table = []
wikidataEndpointUrl = 'https://query.wikidata.org/sparql'
acceptMediaType = 'application/json'
r = requests.get(wikidataEndpointUrl, params={'query' : query}, headers = generateHeaderDictionary(acceptMediaType))
try:
    data = r.json()
    statements = data['results']['bindings']
    for statement in statements:
        unit_iri = statement['unit']['value']
        unit_qnumber = extract_qnumber(unit_iri)
        parent_iri = statement['parent']['value']
        parent_qnumber = extract_qnumber(parent_iri)
        unit_label = statement['label']['value']
        unit_table.append({'unit': unit_qnumber, 'label': unit_label, 'parent': parent_qnumber})
except:
    unit_table = [r.text]
# delay a quarter second to avoid hitting the SPARQL endpoint to rapidly
#sleep(sparqlSleep)

#print(json.dumps(unit_table, indent=2))

writeDictsToCsv(unit_table, 'vanderbilt_units.csv', ['unit', 'label', 'parent'])


# create blank files for institution-wide data

unit_files = ['vu_total.csv', 'vu_men.csv', 'vu_works.csv', 'vu_women.csv', 'vu_orcid.csv', 'vu_men_works.csv', 'vu_women_works.csv']

for file_name in unit_files:
    header_row = ['date', 'count']
    header_table = [header_row]
    writeListsToCsv(file_name, header_table)

# create blank files for units data

unit_files = ['units_total.csv', 'units_women.csv', 'units_men.csv', 'units_orcid.csv', 'units_works.csv', 'units_works_men.csv', 'units_works_women.csv']

for file_name in unit_files:
    header_row = ['date']
    for unit in unit_table:
        header_row.append(unit['unit'])
    header_table = [header_row]
    writeListsToCsv(file_name, header_table)
    
# Write date when last run
with open('last_run.txt', 'wt', encoding='utf-8') as fileObject:
    fileObject.write(generate_utc_date())

print('done')

done


In [ ]:
# Script to actually collect the data

#data = readDict('vanderbilt_units.csv')
#print(json.dumps(data,indent=2))

while True: # infinite loop
    print('Time checked:', datetime.datetime.utcnow().isoformat())
    with open('last_run.txt', 'rt', encoding='utf-8') as fileObject:
        date_last_run = fileObject.read()
    print('Date last run:', date_last_run)

    date_now_utc = generate_utc_date()
    print('UTC date now is:', date_now_utc)

    if date_now_utc > date_last_run:
        run_all_queries()

        # Update the date last run
        with open('last_run.txt', 'wt', encoding='utf-8') as fileObject:
            fileObject.write(generate_utc_date())

        print('done')
    print()

    # wait an hour before checking again
    sleep(3600)

Time checked: 2020-05-05T23:38:14.765565
Date last run: 2020-05-04
UTC date now is: 2020-05-05
vu_total.csv
vu_men.csv
vu_women.csv
vu_orcid.csv
vu_works.csv
vu_men_works.csv
vu_women_works.csv
units_total.csv
units_women.csv
units_men.csv
units_orcid.csv
units_works.csv
units_works_men.csv
units_works_women.csv
done

